<a href="https://colab.research.google.com/github/noahvlone/sistem-pendukung-keputusan/blob/main/Sistem_Pendukung_Keputusan_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("NIM         : 4522210103")
print("Nama        : Farhan Ramadhan")
print("Mata Kuliah : Sistem Pendukung Keputusan A")

NIM         : 4522210103
Nama        : Farhan Ramadhan
Mata Kuliah : Sistem Pendukung Keputusan A


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
# Fungsi untuk menghitung bobot dari matriks perbandingan berpasangan
def hitung_bobot(matriks):
    total_per_kolom = np.sum(matriks, axis=0)
    matriks_normalisasi = matriks / total_per_kolom
    bobot = np.mean(matriks_normalisasi, axis=1)
    return bobot

In [ ]:
# Fungsi untuk menghitung Rasio Konsistensi (CI / RI)
def hitung_konsistensi(matriks, bobot):
    lambdamax = np.mean(np.sum(matriks * bobot, axis=1) / bobot)
    CI = (lambdamax - len(bobot)) / (len(bobot) - 1)
    # Nilai Acak Konsistensi (RI) untuk ukuran matriks 10x10 adalah 1.49
    RI = 1.49
    CR = CI / RI
    return CR

In [ ]:
# Mengambil data historis dari Yahoo Finance
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMD', 'AMZN', 'META', 'NVDA', 'BABA', 'NFLX']
data = yf.download(symbols, start='2024-11-01', end='2024-11-07')
data.head()

[*********************100%***********************]  10 of 10 completed


Price                       Adj Close                                     \
Ticker                           AAPL         AMD        AMZN       BABA   
Date                                                                       
2024-11-01 00:00:00+00:00  222.665024  141.860001  197.929993  97.580002   
2024-11-04 00:00:00+00:00  221.766006  140.710007  195.779999  98.400002   
2024-11-05 00:00:00+00:00  223.204422  141.660004  199.500000  99.209999   
2024-11-06 00:00:00+00:00  222.475235  145.100006  207.089996  96.730003   

Price                                                                      \
Ticker                          GOOGL        META        MSFT        NFLX   
Date                                                                        
2024-11-01 00:00:00+00:00  171.289993  567.159973  410.369995  756.099976   
2024-11-04 00:00:00+00:00  169.240005  560.679993  408.459991  755.510010   
2024-11-05 00:00:00+00:00  169.740005  572.429993  411.459991  763.909973   
2024-11-06 00:00:00+00:00  176.509995  572.049988  420.179993  780.210022   

Price                                              ...    Volume            \
Ticker                           NVDA        TSLA  ...      AAPL       AMD   
Date                                               ...                       
2024-11-01 00:00:00+00:00  135.399994  248.979996  ...  65276700  39027400   
2024-11-04 00:00:00+00:00  136.050003  242.839996  ...  44944500  29117400   
2024-11-05 00:00:00+00:00  139.910004  251.440002  ...  28111300  27067300   
2024-11-06 00:00:00+00:00  145.610001  288.529999  ...  54561100  32911500   

Price                                                                        \
Ticker                         AMZN      BABA     GOOGL      META      MSFT   
Date                                                                          
2024-11-01 00:00:00+00:00  99687800   8501000  31796500  15303200  24230400   
2024-11-04 00:00:00+00:00  38492100   8681900  21492700  12064600  19672300   
2024-11-05 00:00:00+00:00  30564800  10804200  18242100   9775400  17626000   
2024-11-06 00:00:00+00:00  72292200  18162900  33695500  18305400  26681800   

Price                                                     
Ticker                        NFLX       NVDA       TSLA  
Date                                                      
2024-11-01 00:00:00+00:00  2996800  207127800   57544800  
2024-11-04 00:00:00+00:00  1902200  187528200   68802400  
2024-11-05 00:00:00+00:00  2200600  160537400   69282500  
2024-11-06 00:00:00+00:00  3690800  242043900  165228700  

[4 rows x 60 columns]

In [ ]:
# Menghitung pengembalian harian dan volatilitas untuk setiap saham
returns = data['Adj Close'].pct_change()
volatilitas = returns.std()
return_harian = returns.mean()
volume_harian = data['Volume'].mean()

In [ ]:
# Matriks Perbandingan Berpasangan untuk Kriteria: Risiko, Return, Likuiditas, Jangka Waktu
matriks_kriteria = np.array([
    [1, 3, 5, 7],
    [1/3, 1, 3, 5],
    [1/5, 1/3, 1, 3],
    [1/7, 1/5, 1/3, 1]
])

# Hitung bobot dan cek konsistensi
bobot_kriteria = hitung_bobot(matriks_kriteria)
CR = hitung_konsistensi(matriks_kriteria, bobot_kriteria)
print("Bobot Kriteria:", bobot_kriteria)
print("Rasio Konsistensi:", CR)

Bobot Kriteria: [0.55789248 0.26334511 0.12187261 0.0568898 ]
Rasio Konsistensi: 0.026502385941245217


In [ ]:
# Matriks Perbandingan Berpasangan 10x10 untuk Alternatif pada setiap Kriteria
matriks_risiko = np.zeros((10, 10))
matriks_return = np.zeros((10, 10))
matriks_likuiditas = np.zeros((10, 10))

# Matriks Risiko berdasarkan volatilitas
for i in range(10):
    for j in range(10):
        matriks_risiko[i][j] = volatilitas[i] / volatilitas[j]

# Matriks Return berdasarkan pengembalian rata-rata
for i in range(10):
    for j in range(10):
        matriks_return[i][j] = return_harian[i] / return_harian[j]

# Matriks Likuiditas berdasarkan volume perdagangan
for i in range(10):
    for j in range(10):
        matriks_likuiditas[i][j] = volume_harian[i] / volume_harian[j]

<ipython-input-33-f4883a18e6c3>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  matriks_risiko[i][j] = volatilitas[i] / volatilitas[j]
<ipython-input-33-f4883a18e6c3>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  matriks_return[i][j] = return_harian[i] / return_harian[j]
<ipython-input-33-f4883a18e6c3>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  matriks_likuiditas[i][j] = volume_harian[i] / volume_harian[j]


In [ ]:
# Hitung bobot untuk setiap alternatif pada setiap kriteria
bobot_risiko = hitung_bobot(matriks_risiko)
bobot_return = hitung_bobot(matriks_return)
bobot_likuiditas = hitung_bobot(matriks_likuiditas)

# Gabungkan bobot kriteria dan alternatif untuk mendapatkan skor akhir
skor_akhir = (
    bobot_kriteria[0] * bobot_risiko +
    bobot_kriteria[1] * bobot_return +
    bobot_kriteria[2] * bobot_likuiditas
)

In [ ]:
# Tampilkan rekomendasi berdasarkan skor akhir
print("\nSkor Akhir untuk setiap alternatif investasi:")
for alt, skor in zip(symbols, skor_akhir):
    print(f"{alt}: {skor}")

# Menentukan Top 3 saham terbaik
top_3_indices = np.argsort(skor_akhir)[-3:][::-1]  # Mengambil 3 skor tertinggi
top_3_symbols = [symbols[i] for i in top_3_indices]
top_3_scores = [skor_akhir[i] for i in top_3_indices]

print("\nTop 3 Saham Terbaik Berdasarkan Analisis:")
for i, (symbol, score) in enumerate(zip(top_3_symbols, top_3_scores), start=1):
    print(f"{i}. {symbol}: {score}")

rekomendasi = symbols[np.argmax(skor_akhir)]
print(f"\nRekomendasi Investasi Terbaik: {rekomendasi}")


Skor Akhir untuk setiap alternatif investasi:
AAPL: 0.02475184523669595
GOOGL: 0.06117160210235899
MSFT: 0.10348742829265968
TSLA: 0.042028237526530274
AMD: 0.08967665397028973
AMZN: 0.0478918059810092
META: 0.05175954772924701
NVDA: 0.04804228513404993
BABA: 0.14085772437749738
NFLX: 0.3334430682745663

Top 3 Saham Terbaik Berdasarkan Analisis:
1. NFLX: 0.3334430682745663
2. BABA: 0.14085772437749738
3. MSFT: 0.10348742829265968

Rekomendasi Investasi Terbaik: NFLX


In [ ]:
# Menyimpan data untuk setiap saham ke file CSV
all_data = {}  # Dictionary untuk menyimpan semua data saham

# Mengunduh data untuk setiap saham
for saham in symbols:
    data = yf.download(saham, start='2024-10-13', end='2024-10-27')
    all_data[saham] = data

# Menyimpan data ke dalam file CSV
for saham, data in all_data.items():
    data.to_csv(f'data_{saham}.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
